# MOVIE RECOMMENDATION USING RESTRICTED BOLTZMANN MACHINE

# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

# Constants

In [2]:
MOVIES_DAT_1M = 'Movielense_Dataset/ml-1m/movies.dat'
USERS_DAT_1M = 'Movielense_Dataset/ml-1m/users.dat'
RATINGS_DAT_1M = 'Movielense_Dataset/ml-1m/ratings.dat'
TRAINING_SET_100K_U1 = 'Movielense_Dataset/ml-100k/u1.base'
TEST_SET_100K_U1 = 'Movielense_Dataset/ml-100k/u1.test'

# Importing the Dataset

In [3]:
colnames = ['MOVIE_ID', 'MOVIE_NAME', 'GENERE']
movies = pd.read_csv(MOVIES_DAT_1M, sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = colnames)
movies.head()

,MOVIE_ID,MOVIE_NAME,GENERE
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
colnames = ['USER_ID', 'GENDER', 'AGE', 'USERS_JOB', 'ZIP_CODE']
users = pd.read_csv(USERS_DAT_1M, sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = colnames)
users.head()

,USER_ID,GENDER,AGE,USERS_JOB,ZIP_CODE
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
colnames = ['USER_ID', 'MOVIE_ID', 'RATINGS', 'TIMESTAMP']
ratings = pd.read_csv(RATINGS_DAT_1M, sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = colnames)
ratings.head()

,USER_ID,MOVIE_ID,RATINGS,TIMESTAMP
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Preparing the training set and the test set

In [6]:
colnames = ['USER_ID', 'MOVIE_ID', 'RATINGS', 'TIMESTAMP']
training_set = pd.read_csv(TRAINING_SET_100K_U1, delimiter = '\t', header = None, names = colnames)
training_set = np.array(training_set, dtype = 'int64') # converting it into numpy array
print('Training Set Shape: ', training_set.shape)

Training Set Shape:  (80000, 4)


In [7]:
colnames = ['USER_ID', 'MOVIE_ID', 'RATINGS', 'TIMESTAMP']
test_set = pd.read_csv(TEST_SET_100K_U1, delimiter = '\t', header = None, names = colnames)
test_set = np.array(test_set, dtype = 'int64') # converting it into numpy array
print('Test Set Shape: ', test_set.shape)

Test Set Shape:  (20000, 4)


# Getting Number of Users and Movies

In [8]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print('Number of Users: ', nb_users)
print('Number of Movies: ', nb_movies)

Number of Users:  943
Number of Movies:  1682


## Converting the data into an array with users in lines and movies in columns

In [9]:
"""converts the data into an array with users in lines and movies in columns"""
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [10]:
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors

In [11]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Converting the ratings into binary ratings 1 (Liked) 0 (Not Liked)

In [12]:
training_set[training_set == 0] = -1 # -1 being, the user haven't rated the movie yet
training_set[training_set == 1] = 0  # the user not liked the movie
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1 # the user liked the movie

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the Architecture of the Neural Network

In [13]:
class RBM():
    """nv - number of visible nodes, nh - number of hidden nodes."""
    def __init__(self, nv, nh):
        self.W = torch.randn(nv, nh) # weights
        self.a = torch.randn(1, nh) # bias : P(h) given v
        self.b = torch.randn(1, nv) # bias : P(v) given h
        
    def sample_h(self, x):
        
        """x - the visible neurons, v in the probabilities p(h) given v."""
    
        wx = torch.mm(x, self.W) # product of x, W (with a transpose to make multiplication consistent)
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        
        """y - the hidden neurons, h in the probabilities p(v) given h."""
    
        wy = torch.mm(y, self.W.t()) # product of y, W
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        
        """The contrastive divergence to aproximate the likelihood gradient\n\n
    v0 - input vector
    vk - visible nodes obtained after k samplings (after k iterations and k contrastive divergence)
    ph0 - vector of probabilities that at the first iteration the hidden nodes equls one given the values of v0
    phk - probabilities of the hidden nodes after k samplings given the values of visible nodes vk."""
    
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [14]:
nv = len(training_set[0])
nh = 100
rbm = RBM(nv, nh)

# Training the RBM

In [15]:
nb_epoch = 10
batch_size = 100
k_steps = 10 # k-steps for contrastive divergence
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0. # counter used in Normalization
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(k_steps):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            # we don't want to learn where there is no rating
            # not include the cells that contains -1 ratings in the training process
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # simple distance between the prediction and the real rating
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s)) # printing normalized train_loss
                                                            # use str((test_loss / s).tolist()) to print as proper floats

epoch: 1 loss: tensor(0.3548)
epoch: 2 loss: tensor(0.2369)
epoch: 3 loss: tensor(0.2525)
epoch: 4 loss: tensor(0.2494)
epoch: 5 loss: tensor(0.2459)
epoch: 6 loss: tensor(0.2483)
epoch: 7 loss: tensor(0.2490)
epoch: 8 loss: tensor(0.2474)
epoch: 9 loss: tensor(0.2500)
epoch: 10 loss: tensor(0.2455)


# Testing the RNN

In [16]:
k_steps = 10 # k-steps for contrastive divergence
test_loss = 0
s = 0. # counter used in Normalization
for id_user in range(0, nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    # one step (iteration) of Gibb's sampling (or blind walk according to this test set) is enough as we've
    # already trained wi k-steps
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # simple distance between the prediction and the real rating
    s += 1.
print('test loss: '+str(test_loss/s)) # printing normalized test_loss
                                      # use str((test_loss / s).tolist()) to print as proper floats

test loss: tensor(0.1129)
